<a href="https://colab.research.google.com/github/Shuraimi/fastai_DL_course/blob/main/IMDb_Sentiment_Analysis(Lesson4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook, I'll be reproducing the chapter 10 i.e. IMDb Sentiment Analysis.

## Steps I'll take
0. Imports
1. Take a look at tokenization
2. Then at numericalization
3. Make data ready or assemble data to train language model i.e. create dataloader
4. Fine-tune the language model on IMDb corpus
5. Fine tune it to get a classifier for sentiment analysis

## Imports

In [ ]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 787.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 206.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires 

In [ ]:
#hide
from fastbook import *
from IPython.display import display,HTML

In [ ]:
from fastai.text.all import *

## Tokenization

First download and extract the data

In [ ]:
path=untar_data(URLs.IMDB)

In [ ]:
# looking at the folders in the path
path.ls()

(#7) [Path('/root/.fastai/data/imdb/test'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/imdb.vocab'),Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/README'),Path('/root/.fastai/data/imdb/tmp_lm'),Path('/root/.fastai/data/imdb/tmp_clas')]

In [ ]:
#get all text files
files=get_text_files(path,folders=['train','test','unsup'])

In [ ]:
# open a file and read it and display a part of it
txt=files[0].open().read()
txt[:50]

'Do-It-Yourself indie horror auteur Todd Sheets ret'

### Word tokenization

For tokenizing, we use `SpacyTokenizer` which was the tokenizer used at first or we can use the `WordTokenizer` by fastai which refers to the default tokenizer at present.

We need to pass the txt as a list.

In [ ]:
spacy=SpacyTokenizer()
tkn=first(spacy([txt]))

In [ ]:
tkn

(#364) ['Do','-','It','-','Yourself','indie','horror','auteur','Todd','Sheets'...]

Or we can use the `WordTokenizer`

In [ ]:
wt=WordTokenizer()
tkns=first(wt([txt]))
tkns

(#364) ['Do','-','It','-','Yourself','indie','horror','auteur','Todd','Sheets'...]

Other than these, fastai provides a `Tokenizer` class which provides few additional functionality to tokenization process such as adding special tokens while tokenization.

We pass the word tokenizer to this class to specify word based tokenization strategy while leveraging the additional functionality of fastai's `Tokenizer`.

In [ ]:
tknr=Tokenizer(wt)
print(coll_repr(tknr(txt),35))

(#379) ['xxbos','do','-','it','-','yourself','indie','horror','auteur','xxmaj','todd','xxmaj','sheets','returns','with','another','entertainingly','atrocious',"nickel'n'dime",'shot','-','on','-','video','clunker','that',"'s",'basically','just','a','feeble','excuse','to','sling','around'...]


In [ ]:
help(Tokenizer)

Help on class Tokenizer in module fastai.text.core:

class Tokenizer(fastcore.transform.Transform)
 |  Tokenizer(self, tok, rules=None, counter=None, lengths=None, mode=None, sep=' ')
 |  
 |  Provides a consistent `Transform` interface to tokenizers operating on `DataFrame`s and folders
 |  
 |  Method resolution order:
 |      Tokenizer
 |      fastcore.transform.Transform
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, tok, rules=None, counter=None, lengths=None, mode=None, sep=' ')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  decodes = (object,object) -> decodes
 |  
 |  encodes = (Path,object) -> encodes
 |  (str,object) -> encodes
 |  
 |  get_lengths(self, items)
 |  
 |  setups = (object,object) -> setups
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  from_df(text_cols, tok=None, rules=None, sep=' ', *, n_workers=2, mark_fields=None, tok_te

### Subword tokenization

We define a function for subword tokenization which
- instantiates the SubwordTokenizer and takes vocab size
- runs setup on txts
- and returns tokens of subwords joined

We grab the 200 files inorder to train the subword tokenizer to find the most commonly occuring group of letters in words.

In [ ]:
txts=L(o.open().read() for o in files[:2000])

In [ ]:
txts[0]

"Do-It-Yourself indie horror auteur Todd Sheets returns with another entertainingly atrocious nickel'n'dime shot-on-video clunker that's basically just a feeble excuse to sling around a lot of watery blood and gleaming guts as often as possible. An evil demonic scarecrow resurrects the dead as ravenous rot-faced zombies so they can feast on the living. A bunch of bickering college kids, a trio of dangerous escaped convicts led by the vicious Slade (Byron Nichodemus hamming it up to an outrageous degree), two equally savage sleazeball hoodlums, and a trio of hottie sisters all have to do their best to survive this harrowing ordeal. That's it for the needlessly muddled and convoluted plot, but fortunately what this hilariously horrendous hoot lacks in narrative coherence (plenty) it more than compensates for with a pleasing plethora of gloriously gross'n'graphic gore. Disgusting highlights include a woman having her fingers chopped off, a fatal gunshot to a young gal's groin, attempted n

In [ ]:
# function for subword tokenization
def subword_tokenization(sz):
    sub=SubwordTokenizer(vocab_sz=sz)
    sub.setup(txts)
    return ' '.join(first(sub([txt])))[:40]

In [ ]:
subword_tokenization(200)

'▁ D o - I t - Y o ur s el f ▁in d i e ▁ '

Now that we have seen how we do tokenization, we need to convert them into numbers.

## Numericalization

1. Create vocab
2. Convert the tokens into numbers

In [ ]:
num=Numericalize()
num.setup(txts)

In [ ]:
numericalized_text=num(txt)

In [ ]:
numericalized_text

TensorText([48, 13, 43,  ..., 17, 10, 30])

Now we map these numbers in vocab to see whether we get the same text as before.

In [ ]:
num.vocab[48]

'D'

In [ ]:
n=[num.vocab[i] for i in numericalized_text]

In [ ]:
n

['D',
 'o',
 '-',
 'I',
 't',
 '-',
 'Y',
 'o',
 'u',
 'r',
 's',
 'e',
 'l',
 'f',
 ' ',
 'i',
 'n',
 'd',
 'i',
 'e',
 ' ',
 'h',
 'o',
 'r',
 'r',
 'o',
 'r',
 ' ',
 'a',
 'u',
 't',
 'e',
 'u',
 'r',
 ' ',
 'T',
 'o',
 'd',
 'd',
 ' ',
 'S',
 'h',
 'e',
 'e',
 't',
 's',
 ' ',
 'r',
 'e',
 't',
 'u',
 'r',
 'n',
 's',
 ' ',
 'w',
 'i',
 't',
 'h',
 ' ',
 'a',
 'n',
 'o',
 't',
 'h',
 'e',
 'r',
 ' ',
 'e',
 'n',
 't',
 'e',
 'r',
 't',
 'a',
 'i',
 'n',
 'i',
 'n',
 'g',
 'l',
 'y',
 ' ',
 'a',
 't',
 'r',
 'o',
 'c',
 'i',
 'o',
 'u',
 's',
 ' ',
 'n',
 'i',
 'c',
 'k',
 'e',
 'l',
 "'",
 'n',
 "'",
 'd',
 'i',
 'm',
 'e',
 ' ',
 's',
 'h',
 'o',
 't',
 '-',
 'o',
 'n',
 '-',
 'v',
 'i',
 'd',
 'e',
 'o',
 ' ',
 'c',
 'l',
 'u',
 'n',
 'k',
 'e',
 'r',
 ' ',
 't',
 'h',
 'a',
 't',
 "'",
 's',
 ' ',
 'b',
 'a',
 's',
 'i',
 'c',
 'a',
 'l',
 'l',
 'y',
 ' ',
 'j',
 'u',
 's',
 't',
 ' ',
 'a',
 ' ',
 'f',
 'e',
 'e',
 'b',
 'l',
 'e',
 ' ',
 'e',
 'x',
 'c',
 'u',
 's',
 'e',
 ' '

In [ ]:
''.join(num.vocab[i] for i in numericalized_text)

"Do-It-Yourself indie horror auteur Todd Sheets returns with another entertainingly atrocious nickel'n'dime shot-on-video clunker that's basically just a feeble excuse to sling around a lot of watery blood and gleaming guts as often as possible. An evil demonic scarecrow resurrects the dead as ravenous rot-faced zombies so they can feast on the living. A bunch of bickering college kids, a trio of dangerous escaped convicts led by the vicious Slade (Byron Nichodemus hamming it up to an outrageous degree), two equally savage sleazeball hoodlums, and a trio of hottie sisters all have to do their best to survive this harrowing ordeal. That's it for the needlessly muddled and convoluted plot, but fortunately what this hilariously horrendous hoot lacks in narrative coherence (plenty) it more than compensates for with a pleasing plethora of gloriously gross'n'graphic gore. Disgusting highlights include a woman having her fingers chopped off, a fatal gunshot to a young gal's groin, attempted n

Now that we numbers, we need to make batches.

## Putting our text into batches to create Language model

Here the process is as follows:-

At each epoch,

- documents are shuffled and sorted
- stream is cut down into batches which consist of mini-batches of fixed size
- and these are passed to the model.??????

This is done by the `LMDataLoader` which creates dependent and independent variables for us but we need to pass it numericalized tokens **had difficulty in LMDataLoaders part**

In [ ]:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."

bs,seq_len=6,15
tokens=tknr(stream)
d_tokens=np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df=pd.DataFrame(d_tokens)

# forgot this line of code
display(HTML( df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [ ]:
toks200=txts[:200].map(tknr)

In [ ]:
nums200=toks200.map(num)

In [ ]:
dl=LMDataLoader(nums200)

In [ ]:
x,y=first(dl)

In [ ]:
x

LMTensorText([[ 2,  0, 43,  ..., 30, 12,  0],
              [ 0,  0, 51,  ...,  0,  0,  0],
              [ 0, 30,  0,  ...,  8,  0,  8],
              ...,
              [ 0,  0,  0,  ...,  0,  0,  0],
              [ 0,  0,  0,  ..., 32,  0,  0],
              [ 0,  0,  0,  ...,  0,  0,  0]])

In [ ]:
# checking dependent variables
' '.join(num.vocab[o] for o in x[0])

'xxbos xxunk - xxunk - xxunk xxunk xxunk xxunk xxmaj xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk - xxunk - xxunk xxunk xxunk xxunk xxunk xxunk a xxunk xxunk xxunk xxunk xxunk a xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk . xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk - xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk . a xxunk'

## Training text classifier

1. language model using datablock
2. fine tune it
3. save and load
4. text generation
5. creating classifier dataloaders
6. fine tuning it

forgot these
```
batch_size=128,seq_len=80

splitter=GrandparentSplitter(0.1) the numebr 0.1
```



In [ ]:
get_imdb=partial(get_text_files,folders=['train','test','unsup'])
lm_dl=DataBlock(
    blocks=TextBlock.from_folder(path,is_lm=True),
                get_items=get_imdb,
                #don;t need since lm has embedded labels in independent variable
                #get_y=parent_label,
                splitter=GrandparentSplitter(0.1)).dataloaders(path,path=path,batch_size=128,seq_len=80)

FileNotFoundError: [Errno 2] No such file or directory: '/root/.fastai/data/imdb_tok/counter.pkl'



```
drop_mult=0.3

.to_fp16()
```
forgot these two


In [ ]:
# now pass this dataloader of language model to language model learner
lm_learner=language_model_learner(lm_dl,AWD_LSTM,drop_mult=0.3,metrics=[accuracy,Perplexity()]).to_fp16()

In [ ]:
# fine tune it
lm_learner.fit_one_cycle(1,2e-2)

### Save and load learner

In [ ]:
# save the 1 epoch learner
lm_learner.save('1epoch')

In [ ]:
# load it
learn=lm_learner('1epoch')

In [ ]:
# now unfreeze the model and fine tune it
learn.unfreeze()
learn.fit_one_cycle(10,2e-3)

Now save the encoder i.e. the body and leave the task specific head

In [ ]:
learn.save_encoder('fine_tuned')

### Text generation

FORGOT what parameters predict takes

In [ ]:
TEXT = "I liked this movie because"
N=22
N_SEN=2
preds=[learn.predict(TEXT, N, temperature=0.7) for _ in range(N_SEN)]

## Creating classifer dataloaders

using DAtaBlock API

forgot to consider vocab of dataloader of language model and not learner

In [ ]:
clas_dl=DataBlock(
    blocks=(TextBlock.from_folder(path,vocab=lm_dl.vocab),CategoryBlock),
    get_y=parent_label,
    get_items=partial(get_text_files,folders=['train','test','unsup']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path,path=path,batch_size=128,seq_len=72)

In [ ]:
clas_dl.show_batch(max_n=2)

### create learner

forgot to_fp16

In [ ]:
learn=text_classifier_learner(clas_dl,AWD_LSTM, metrics=[accuracy],drop_mult=0.5).to_fp16()

Now we have to load thye encoder we saved after fine tuning language model on IMDb corpus(forgot this part

In [ ]:
learn.load_encoder('fine_tuned')

What this does is that it the encoder part of the learn(text_classifier_learner) is replaced with that of language model encoder.

## Fine tuning classifier

first we fine tune once using fit_one_cycle

then freeze all layers excpet the last 2 layers and freeze all excpet last three layers and then at last unfreeeze all layers

In [ ]:
learn.fit_one_cycle(1,2e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1,slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1,slice(5e-2/(2.6**4),5e-2))

In [ ]:
# unfreeze whole model
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))